In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import allmusicUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from artists import artists
from artistsAM import artistsAM
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-02-19 21:24:14.415266


In [2]:
%load_ext autoreload
%autoreload
disc   = discogs('allmusic')
dutils = allmusicUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Artists From Search

In [ ]:
192   Astronaut Ape                           set()
789   Color me Badd                           set()
1086  Dr.Alban                                set()
1087  Dyango                                  set()
1232  Elba Ramalho                            set()
1389  Fabolous and Jadakiss                   set()
1390  Frankie Valli & The Four Seasons        {'Frankie Valli & The Romans'}
1554  Gary Lewis                              {'Gary Lucas'}
1663  Heart                                   set()
1936  Jaded Heart                             set()
2509  Marco Masini                            {'Micro Mini'}
2510  Michael Franti & Spearhead              set()
2722  Oresund Space Collective                set()
3481  Social Distortion                       set()
3711  Tom Keifer                              set()
4258  The Whispers                            {'The Whites'}

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs('a')
arts = artists(disc)
arts.searchDiscogForArtist("The Whispers")

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

In [ ]:
def getURLData(url, debug=True):    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading: {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
    
    return data, response

In [ ]:
myMusicMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [ ]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

In [ ]:
from json import loads
baseURL = "https://www.allmusic.com/search/artists/"
for i,(artistName,artistVals) in enumerate(myMusicMap.items()):
    if i < 3871:
        continue
    print(i,'/',len(myMusicMap),'  \t',artistName)
    url     = urllib.parse.urljoin(baseURL, quote(artistName))

    data, response = getURLData(url)
    if response.code == 200:
        bsdata = getHTML(data)
    else:
        continue
        
    infos = bsdata.findAll("div", {"class": "info"})
    for info in infos:
        for name in info.findAll("div", {"class": "name"}):
            ref = name.find("a")
            if ref is None:
                continue
            href     = ref.attrs['href']
            artistID = loads(ref.attrs['data-tooltip'])['id']
            name     = ref.text
            if allMusicIDMap.get(artistID) is None:
                allMusicIDMap[artistID] = [name, href]
                print("\t{0: <70}{1: <20}".format(name,artistID))
                
    print("  -->",len(allMusicIDMap))
    sleep(2)
    if i % 50 == 0:
        savename = "collections/allMusicIDMap.p"
        print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
        saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

In [ ]:

savename = "collections/allMusicIDMap.p"
print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

In [ ]:
allMusicIDMap['MN0000961234']

In [ ]:
amInfo  = allMusicIDMap['MN0000961234']
url     = "{0}/discography/all".format(amInfo[1])
data, response = getURLData(url, debug=True)

In [ ]:
savename = "MN0000961234.p"
force = True
print("Saving {0} (force={1})".format(savename, force))
saveJoblib(data=data, filename=savename, compress=True)

In [ ]:
data = getFile(savename)

In [ ]:
url = "https://www.allmusic.com/artist/dave-matthews-band-mn0000961234/discography/all"

# All Music

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [ ]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

In [6]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

56646

In [16]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available


In [13]:
allMusicIDMap['MN0000986327']

['Billy Lee Myers, Jr.',
 'https://www.allmusic.com/artist/billy-lee-myers-jr-mn0000986327']

In [19]:
from time import sleep
for i,(artistID,artistVals) in enumerate(allMusicIDMap.items()):
    if i <= 49496:
        continue
    artistName,ref = artistVals
    print(i,'/',len(allMusicIDMap),'\t',artistName,'\t',ref)
    url      = artsAM.getArtistURL(ref)
    savename = artsAM.getArtistSavename(artistID)
    artsAM.downloadArtistURL(url, savename, force=True)

49497 / 56646 	 Maze of Time 	 https://www.allmusic.com/artist/maze-of-time-mn0001541108
Saving /Users/tgadfort/Music/Discog/artists-allmusic/8/0001541108.p (force=True)
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
49498 / 56646 	 Maze of Sound 	 https://www.allmusic.com/artist/maze-of-sound-mn0003382995
Saving /Users/tgadfort/Music/Discog/artists-allmusic/95/0003382995.p (force=True)
  --> This file is 16.1kB.
Done. Sleeping for 2 seconds
49499 / 56646 	 Maze of Terror 	 https://www.allmusic.com/artist/maze-of-terror-mn0003494839
Saving /Users/tgadfort/Music/Discog/artists-allmusic/39/0003494839.p (force=True)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
49500 / 56646 	 The Horde of Torment 	 https://www.allmusic.com/artist/the-horde-of-torment-mn0003612148
Saving /Users/tgadfort/Music/Discog/artists-allmusic/48/0003612148.p (force=True)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
49501 / 56646 	 God of Torment 	 https://www.allmusic.com/artist/god-of

Saving /Users/tgadfort/Music/Discog/artists-allmusic/99/0001179499.p (force=True)
  --> This file is 18.3kB.
Done. Sleeping for 2 seconds
49531 / 56646 	 Jones McGinnis 	 https://www.allmusic.com/artist/jones-mcginnis-mn0002426687
Saving /Users/tgadfort/Music/Discog/artists-allmusic/87/0002426687.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
49532 / 56646 	 John Mick 	 https://www.allmusic.com/artist/john-mick-mn0003235220
Saving /Users/tgadfort/Music/Discog/artists-allmusic/20/0003235220.p (force=True)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
49533 / 56646 	 Gene Mick 	 https://www.allmusic.com/artist/gene-mick-mn0003406691
Saving /Users/tgadfort/Music/Discog/artists-allmusic/91/0003406691.p (force=True)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
49534 / 56646 	 Mic Jones 	 https://www.allmusic.com/artist/mic-jones-mn0000382497
Saving /Users/tgadfort/Music/Discog/artists-allmusic/97/0000382497.p (force=True)
  --> This file is 15.2k

49565 / 56646 	 Mike West 	 https://www.allmusic.com/artist/mike-west-mn0002031853
Saving /Users/tgadfort/Music/Discog/artists-allmusic/53/0002031853.p (force=True)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
49566 / 56646 	 Mike West 	 https://www.allmusic.com/artist/mike-west-mn0002481190
Saving /Users/tgadfort/Music/Discog/artists-allmusic/90/0002481190.p (force=True)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
49567 / 56646 	 Mike West 	 https://www.allmusic.com/artist/mike-west-mn0003244476
Saving /Users/tgadfort/Music/Discog/artists-allmusic/76/0003244476.p (force=True)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
49568 / 56646 	 Mike West 	 https://www.allmusic.com/artist/mike-west-mn0003314072
Saving /Users/tgadfort/Music/Discog/artists-allmusic/72/0003314072.p (force=True)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
49569 / 56646 	 Mike Lennon  Marc West 	 https://www.allmusic.com/artist/mike-lennon-marc-west-mn0000889140
Saving /

49601 / 56646 	 Thomas Milhouse 	 https://www.allmusic.com/artist/thomas-milhouse-mn0003834840
Saving /Users/tgadfort/Music/Discog/artists-allmusic/40/0003834840.p (force=True)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
49602 / 56646 	 Mingus & Milhouse 	 https://www.allmusic.com/artist/mingus-milhouse-mn0001993779
Saving /Users/tgadfort/Music/Discog/artists-allmusic/79/0001993779.p (force=True)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
49603 / 56646 	 Psalmist Carol A. Milhouse 	 https://www.allmusic.com/artist/psalmist-carol-a-milhouse-mn0003630509
Saving /Users/tgadfort/Music/Discog/artists-allmusic/9/0003630509.p (force=True)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
49604 / 56646 	 Myllhouse 	 https://www.allmusic.com/artist/myllhouse-mn0000402361
Saving /Users/tgadfort/Music/Discog/artists-allmusic/61/0000402361.p (force=True)
  --> This file is 16.1kB.
Done. Sleeping for 2 seconds
49605 / 56646 	 Millhouse 	 https://www.allmusic.com/artis

49637 / 56646 	 Mirza Schaffy 	 https://www.allmusic.com/artist/mirza-schaffy-mn0001807346
Saving /Users/tgadfort/Music/Discog/artists-allmusic/46/0001807346.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
49638 / 56646 	 Mirza Ghalib 	 https://www.allmusic.com/artist/mirza-ghalib-mn0001890287
Saving /Users/tgadfort/Music/Discog/artists-allmusic/87/0001890287.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
49639 / 56646 	 Mirza Ramic 	 https://www.allmusic.com/artist/mirza-ramic-mn0002103498
Saving /Users/tgadfort/Music/Discog/artists-allmusic/98/0002103498.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
49640 / 56646 	 Mirzâ Abdollâh 	 https://www.allmusic.com/artist/mirz%C3%A2-abdoll%C3%A2h-mn0002581926
Saving /Users/tgadfort/Music/Discog/artists-allmusic/26/0002581926.p (force=True)
  --> This file is 15.1kB.
Done. Sleeping for 2 seconds
49641 / 56646 	 Mirza Bajramovic 	 https://www.allmusic.com/artist/mirza-bajra

...

56475 / 56646 	 Khalid Yasin 	 https://www.allmusic.com/artist/khalid-yasin-mn0001218215
Saving /Users/tgadfort/Music/Discog/artists-allmusic/15/0001218215.p (force=True)
  --> This file is 15.1kB.
Done. Sleeping for 2 seconds
56476 / 56646 	 Ava Max 	 https://www.allmusic.com/artist/ava-max-mn0003655084
Saving /Users/tgadfort/Music/Discog/artists-allmusic/84/0003655084.p (force=True)
  --> This file is 17.5kB.
Done. Sleeping for 2 seconds
56477 / 56646 	 Ava Luna 	 https://www.allmusic.com/artist/ava-luna-mn0002020517
Saving /Users/tgadfort/Music/Discog/artists-allmusic/17/0002020517.p (force=True)
  --> This file is 18.9kB.
Done. Sleeping for 2 seconds
56478 / 56646 	 Max Lorenz 	 https://www.allmusic.com/artist/max-lorenz-mn0002182972
Saving /Users/tgadfort/Music/Discog/artists-allmusic/72/0002182972.p (force=True)
  --> This file is 21.0kB.
Done. Sleeping for 2 seconds
56479 / 56646 	 Max Collie 	 https://www.allmusic.com/artist/max-collie-mn0000331923
Saving /Users/tgadfort/Music/

56513 / 56646 	 Isobel Dacombe 	 https://www.allmusic.com/artist/isobel-dacombe-mn0002034306
Saving /Users/tgadfort/Music/Discog/artists-allmusic/6/0002034306.p (force=True)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
56514 / 56646 	 Abelardo Tucumbi 	 https://www.allmusic.com/artist/abelardo-tucumbi-mn0002458186
Saving /Users/tgadfort/Music/Discog/artists-allmusic/86/0002458186.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
56515 / 56646 	 Oli Dacombe 	 https://www.allmusic.com/artist/oli-dacombe-mn0003086357
Saving /Users/tgadfort/Music/Discog/artists-allmusic/57/0003086357.p (force=True)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
56516 / 56646 	 Liam Dackombe 	 https://www.allmusic.com/artist/liam-dackombe-mn0003130974
Saving /Users/tgadfort/Music/Discog/artists-allmusic/74/0003130974.p (force=True)
  --> This file is 15.4kB.
Done. Sleeping for 2 seconds
56517 / 56646 	 Paul Dackombe 	 https://www.allmusic.com/artist/paul-dackombe-mn0

56549 / 56646 	 N.F. Da Silva Júnior 	 https://www.allmusic.com/artist/nf-da-silva-j%C3%BAnior-mn0003889595
Saving /Users/tgadfort/Music/Discog/artists-allmusic/95/0003889595.p (force=True)
  --> This file is 14.4kB.
Done. Sleeping for 2 seconds
56550 / 56646 	 The Knife 	 https://www.allmusic.com/artist/the-knife-mn0000666244
Saving /Users/tgadfort/Music/Discog/artists-allmusic/44/0000666244.p (force=True)
  --> This file is 21.0kB.
Done. Sleeping for 2 seconds
56551 / 56646 	 Tito Nieves 	 https://www.allmusic.com/artist/tito-nieves-mn0000507558
Saving /Users/tgadfort/Music/Discog/artists-allmusic/58/0000507558.p (force=True)
  --> This file is 27.5kB.
Done. Sleeping for 2 seconds
56552 / 56646 	 Shonen Knife 	 https://www.allmusic.com/artist/shonen-knife-mn0000036310
Saving /Users/tgadfort/Music/Discog/artists-allmusic/10/0000036310.p (force=True)
  --> This file is 26.3kB.
Done. Sleeping for 2 seconds
56553 / 56646 	 NAV 	 https://www.allmusic.com/artist/nav-mn0003569704
Saving /Us

56586 / 56646 	 G. Paul Sullivan 	 https://www.allmusic.com/artist/g-paul-sullivan-mn0001207400
Saving /Users/tgadfort/Music/Discog/artists-allmusic/0/0001207400.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
56587 / 56646 	 G Pal Productions 	 https://www.allmusic.com/artist/g-pal-productions-mn0001279913
Saving /Users/tgadfort/Music/Discog/artists-allmusic/13/0001279913.p (force=True)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
56588 / 56646 	 G. Paul Naeger 	 https://www.allmusic.com/artist/g-paul-naeger-mn0001660921
Saving /Users/tgadfort/Music/Discog/artists-allmusic/21/0001660921.p (force=True)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
56589 / 56646 	 G. Paul Martin 	 https://www.allmusic.com/artist/g-paul-martin-mn0001661279
Saving /Users/tgadfort/Music/Discog/artists-allmusic/79/0001661279.p (force=True)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
56590 / 56646 	 G. Paul Minor 	 https://www.allmusic.com/artist/g-paul

56622 / 56646 	 Los Beatniks 	 https://www.allmusic.com/artist/los-beatniks-mn0001466862
Saving /Users/tgadfort/Music/Discog/artists-allmusic/62/0001466862.p (force=True)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
56623 / 56646 	 The Cappuccino Beatniks 	 https://www.allmusic.com/artist/the-cappuccino-beatniks-mn0001550412
Saving /Users/tgadfort/Music/Discog/artists-allmusic/12/0001550412.p (force=True)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
56624 / 56646 	 Dreamtime Beatniks 	 https://www.allmusic.com/artist/dreamtime-beatniks-mn0001942126
Saving /Users/tgadfort/Music/Discog/artists-allmusic/26/0001942126.p (force=True)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds
56625 / 56646 	 Rodeo Beatniks 	 https://www.allmusic.com/artist/rodeo-beatniks-mn0001968974
Saving /Users/tgadfort/Music/Discog/artists-allmusic/74/0001968974.p (force=True)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds
56626 / 56646 	 Electric Beatniks 	 https://www.allmus

In [ ]:
data = artsAM.getData('/Users/tgadfort/Music/Discog/artists-allmusic/62/0000567862.p')

In [ ]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
file = "MN0000961234.p"
file = "/Users/tgadfort/Music/Discog/artists-allmusic/27/0000986327.p"
artAM = artistAM()
data = artAM.getData(file)

In [ ]:
data

In [ ]:
print("Artist")
info = data.artist.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("URL")
info = data.url.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("ID")
info = data.ID.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Pages")
info = data.pages.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Profile")
info = data.profile.get()
for k,v in info.items():
    if k == "profile":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)
    elif k == "members":
        print("\t",k)
        for v2 in v:
            print("\t\t",v2)
    else:
        print('\t',k,'\t',v)
            

print("Media")
info = data.media.get()
for k,v in info.items():
    if k == "media":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)


print("MediaCounts")
info = data.mediaCounts.get()
for k,v in info.items():
    print("\t",k,'\t',v)